## Day 7 

In [1]:
using DelimitedFiles

In [7]:
rndict = Dict(1 => 3, 2 => 3, 
              3 => 1, 4 => 1, 
              5 => 2, 6 => 2, 
              7 => 3, 8 => 3,
              9 => 1,
              99 => 0)

struct OpCode
    instr::Int
    mode::Array{Int, 1}
    N::Int
end

function OpCode(c::Int)
    digs = digits(c)
    oc = digs[1] + (length(digs) > 1 ? 10*digs[2] : 0)
    N = rndict[oc]
    mode = zeros(Int, N)
    mode[1:length(digs[3:end])] = digs[3:end]
    return OpCode(oc, mode, N)
end 


function intcode_computer(program::Array{Int,1}, input::Channel{Int}, output::Channel{Union{Int, Symbol}}; pc::Int=1)
    
    Np = length(program)
    Np_new = 10*Np
    for j=1:(Np_new-Np)
        push!(program, 0)
    end
    
    jump_flag = false
    rel_base = 0
    
    function load(addr::Int, mode::Int)::Int
        #println("Load: $(addr) -- $(mode)")
        if mode == 0
            return program[addr+1]
        elseif mode == 1
            return addr
        elseif mode == 2
            return program[addr + rel_base + 1]
        else
            error("Unknown mode $(mode).")
        end
    end
    
    function store(data::Int, addr::Int, mode::Int)
        #println("Store: $(data) --> $(addr) -- $(mode)")
        if mode == 0
            program[addr+1] = data
        elseif mode == 1
            error("Invalid mode 1 for store instruction.")
        elseif mode == 2
            program[addr + rel_base + 1] = data
        else
            error("Unknown mode $(mode).")
        end
    end
    
    function load_input(dst::Int, mode::Int)
        #println("Waiting for next input: store to $(dst) -- $(mode)")
        inp = take!(input)
        store(inp, dst, mode)
    end
    
    function store_output(data::Int)
        #println("Tried to store output $(data)")
        put!(output, data)    
    end    
    
    function jtrue(x::Int, y::Int)
        if x != 0 
            pc = y+1
            jump_flag = true
        end
    end
    
    function jfalse(x::Int, y::Int)
        if x == 0  
            pc = y+1 
            jump_flag = true
        end
    end
    
    lt(x::Int, y::Int)::Int = Int(x < y)
    
    eq(x::Int, y::Int)::Int = Int(x == y)
    
    while program[pc] != 99
        jump_flag = false
        oc = OpCode(program[pc])     
        #println(oc)
        if oc.instr == 1
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            store(x+y, program[pc+3], oc.mode[3])
            
        elseif oc.instr == 2
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            store(x*y, program[pc+3], oc.mode[3])
            
        elseif oc.instr == 3
            load_input(program[pc+1], oc.mode[1])
            
        elseif oc.instr == 4
            x = load(program[pc+1], oc.mode[1])
            store_output(x)
            
        elseif oc.instr == 5
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            jtrue(x, y)
        elseif oc.instr == 6
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            jfalse(x, y)
            
        elseif oc.instr == 7
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            store(lt(x,y), program[pc+3], oc.mode[3])
            
        elseif oc.instr == 8
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            store(eq(x,y), program[pc+3], oc.mode[3])
            
        elseif oc.instr == 9
            rel_base += load(program[pc+1], oc.mode[1])
            
        
        else
            @error "Something went wrong! Got code $(oc.instr)."
        end
        
        pc += (jump_flag ? 0 : oc.N+1)
    end
    println("Computer finished!")
    put!(output, :done);
end

intcode_computer (generic function with 1 method)

In [3]:
include("./intcode.jl")

intcode_computer (generic function with 1 method)

In [4]:
left = [[0 -1]; [1 0]]
right = [[0 1]; [-1 0]]

function run_computer(program::Array{Int, 1}; start_color=0)
    
    panels = Dict()
    robot_pos = [0,0]
    robot_dir = [0,1]
    painted = 0
    
    function next_input(dir)
        if dir == 0
            robot_dir = left * robot_dir
        elseif dir == 1
            robot_dir = right * robot_dir
        else
            error("Unknown direction $(dir)!")
        end
        
        robot_pos += robot_dir;
        if robot_pos in keys(panels)
            return panels[robot_pos]
        else
            return 0
        end
    end
    
    input_chan = Channel{Int}(4)
    output_chan = Channel{Union{Int, Symbol}}(4)
    
    comp_task = @async intcode_computer(copy(program), input_chan, output_chan)        
    
    put!(input_chan, start_color)
    
    idx = 0
    done = false
    
    @sync begin
    while !done
        idx += 1
        if isopen(output_chan)
            paint_color = take!(output_chan)
            if paint_color == :done
                println("Got finish code!")
                break
            end
        else
            println("channel Was closed!")
            break
        end
        if !in(robot_pos, keys(panels))
            painted += 1
        end
        panels[robot_pos] = paint_color
        if isopen(output_chan)
            direction = take!(output_chan)
            if paint_color == :done
                println("Got finish code!")
                break
            end
        else
            println("Output chan closed!")
            break
        end
        if isopen(input_chan)
            ni = next_input(direction)
            put!(input_chan, ni)
        else
            break
        end
        
        #print(panels)
        
    end
    end
    
    print(istaskdone(comp_task))
    
    return painted, panels
end

run_computer (generic function with 1 method)

In [5]:
input = readdlm("inputs/day11.txt", ',', Int)[:];

In [6]:
painted, panels = run_computer(input,start_color=0)

Computer finished!
Got finish code!
true

(2088, Dict{Any,Any}([4, 16] => 1,[-21, -16] => 0,[-19, 18] => 1,[-37, -26] => 1,[-7, -9] => 0,[-40, -22] => 0,[-34, -20] => 1,[-5, 20] => 0,[-4, 0] => 1,[-17, 27] => 1…))

In [9]:
draw_panels(panels)

████████████████████████████████████████████████████████████████░░█
███████████████████████████████████████████████████████████████░██░
██████████████████████████████████████████████████████████████░████
████████████████████████████████████████████████████████████████░░░
████████████████████████████████████████████████████████░░░░███░░░░
███████████████████████████████████████████████████████░████░░░█░█░
██████████████████████████████████████████████████████░█░░░░██░░░██
██████████████████████████████████████████████████████░█░░███████░█
██████████████████████████████████████████████████████░░██░█░░░░░██
████████████████████████████████████████████████░░███░░░░█░░███████
████████████████████████████████████████████░░█░█░█░███████░███████
███████████████████████████████████████████░█░░░░███░░█░█░█░███████
███████████████████████████████████████░░██░██░█░░███░███░░░███████
███████████████████████████████████████░░█░███░░██████░█░░█████████
████████████████████████████████████████░░██░██░

In [10]:
painted, panels = run_computer(input, start_color=1)

Computer finished!
Got finish code!
true

(249, Dict{Any,Any}([20, 0] => 0,[20, -2] => 0,[24, -1] => 0,[33, 0] => 1,[34, -4] => 1,[30, -4] => 0,[42, -1] => 0,[20, -5] => 0,[7, -2] => 0,[21, -1] => 1…))

In [11]:
draw_panels(panels)

█░██░█░░░███░░███░░██░░░░█░█████░░██░░░███
█░██░█░██░█░██░█░██░█░████░████░██░█░██░██
█░██░█░██░█░████░██░█░░░██░████░████░██░██
█░██░█░░░██░████░░░░█░████░████░████░░░███
█░██░█░█░██░██░█░██░█░████░████░██░█░█████
██░░██░██░██░░██░██░█░████░░░░██░░██░█████


In [8]:
function draw_panels(panels)
    
    x = [k[1] for k in keys(panels)]
    y = [k[2] for k in keys(panels)]

    nx = abs(maximum(x) - minimum(x))
    ny = abs(maximum(y) - minimum(y))+1
    
    mx = minimum(x)
    my = maximum(y)
    
    #println(nx)
    #println(ny)
    
    #println(mx)
    #println(my)
    draw(x) = x == 0 ? print("█") : print("░")
    
    for j = 1:ny
        for k=1:nx
            #println((mx+k-1,my-j+1))
            pos = [mx+k-1, my-j+1]
            if pos in keys(panels)
                draw(panels[pos])
            else
                draw(0)
                #continue
            end
        end
        print("\n")
    end
end

draw_panels (generic function with 1 method)